In [ ]:
import csv
from datetime import datetime

# ---------- Clases ----------
class Usuario:
    def __init__(self, nombre, apellido, documento, tipo_vinculo, precio_tiquete):
        self.nombre = nombre
        self.apellido = apellido
        self.documento = documento
        self.tipo_vinculo = tipo_vinculo
        self.precio_tiquete = precio_tiquete

    def nombre_completo(self):
        return f"{self.nombre} {self.apellido}"

class Reserva:
    def __init__(self, documento_usuario, dia, hora, pelicula, asiento, precio, fecha_reserva):
        self.documento_usuario = documento_usuario
        self.dia = dia
        self.hora = hora
        self.pelicula = pelicula
        self.asiento = asiento
        self.precio = precio
        self.fecha_reserva = fecha_reserva  # string

# ---------- Cinema ----------
class Cinema:
    def __init__(self, nombre="CineLab UDEA"):
        self.nombre = nombre
        self.usuarios = []     # lista de Usuario
        self.reservas = []     # lista de Reserva

        self.dias = ["Viernes", "Sabado", "Domingo"]
        # Peliculas y horarios fijos
        self.peliculas = [
            ("Interstellar", "2:00pm"),
            ("Oppenheimer", "4:00pm"),
            ("The Imitation Game", "6:00pm")
        ]

        self.funciones = {}    # clave -> mapa 11x11 con "0"/"X"
        self.inicializar_funciones()

        # admin simple en código
        self.admins = {"admin": "1234"}

        # precios por vínculo
        self.vinculos = {
            "1": ("Estudiante", 7500),
            "2": ("Docente", 10000),
            "3": ("Administrativo", 8500),
            "4": ("Oficial interno", 7000),
            "5": ("Publico externo", 15000)
        }

    # ---------- inicializadores ----------
    def inicializar_funciones(self):
        for dia in self.dias:
            for pelicula, hora in self.peliculas:
                clave = self.key_from(dia, hora, pelicula)
                self.funciones[clave] = self.crear_mapa_asientos()

    def crear_mapa_asientos(self):
        return [["0" for _ in range(11)] for _ in range(11)]

    # ---------- utilidades de validacion ----------
    def contiene_digitos(self, texto):
        return any(ch.isdigit() for ch in texto)

    def validar_nombre_apellido(self, valor):
        valor_str = valor.strip()
        if len(valor_str) < 3:
            return False
        if self.contiene_digitos(valor_str):
            return False
        return True

    def validar_documento(self, doc):
        if not doc.isdigit():
            return False
        if not (3 <= len(doc) <= 15):
            return False
        return True

    def validar_asiento_str(self, asiento):
        if len(asiento) < 2:
            return False
        asiento = asiento.strip().upper()
        fila = asiento[0]
        col = asiento[1:]
        if fila < 'A' or fila > 'K':
            return False
        if not col.isdigit():
            return False
        coln = int(col)
        if coln < 1 or coln > 11:
            return False
        return True

    def asiento_to_indices(self, asiento):
        asiento = asiento.strip().upper()
        fila = asiento[0]
        col = int(asiento[1:])
        fila_idx = ord(fila) - ord('A')
        col_idx = col - 1
        return fila_idx, col_idx

    def indices_to_asiento(self, fila_idx, col_idx):
        fila = chr(ord('A') + fila_idx)
        col = col_idx + 1
        return f"{fila}{col}"

    def key_from(self, dia, hora, pelicula):
        # clave uniforme sin espacios en pelicula
        return f"{dia}_{hora}_{pelicula.replace(' ', '')}"

    def contar_asientos_libres(self, mapa):
        return sum(1 for r in mapa for c in r if c == "0")

    # ---------- imprimir mapa ----------
    def imprimir_mapa(self, mapa):
        # encabezado columnas
        encabezado_cols = "   " + " ".join(f"{i:>2}" for i in range(1, 12))
        print(encabezado_cols)
        print("   " + "-" * (3 * 11))
        for i, fila in enumerate(mapa):
            letra = chr(ord('A') + i)
            fila_str = " ".join(f"{c:>2}" for c in fila)
            print(f"{letra} | {fila_str}")
        print()  # linea en blanco

    # ---------- busquedas ----------
    def buscar_usuario_por_documento(self, documento):
        for u in self.usuarios:
            if u.documento == documento:
                return u
        return None

    def buscar_reservas_por_documento(self, documento):
        return [r for r in self.reservas if r.documento_usuario == documento]

    # ---------- registrar usuario ----------
    def registrar_usuario(self):
        print("\n--- Registrar Usuario ---")
        while True:
            nombre = input("Nombre: ").strip().title()
            if not self.validar_nombre_apellido(nombre):
                print("Error: El nombre debe tener al menos 3 letras y no contener números.")
                continue
            break
        while True:
            apellido = input("Apellido: ").strip().title()
            if not self.validar_nombre_apellido(apellido):
                print("Error: El apellido debe tener al menos 3 letras y no contener números.")
                continue
            break
        while True:
            documento = input("Documento (solo números, 3-15 dígitos): ").strip()
            if not self.validar_documento(documento):
                print("Error: Documento inválido. Debe tener solo números y entre 3 y 15 dígitos.")
                continue
            if self.buscar_usuario_por_documento(documento) is not None:
                print("Error: Ya existe un usuario con ese documento.")
                return  # abortar registro
            break

        # mostrar tipos de vínculo
        print("Tipo de vínculo:")
        for k, (nombre_v, precio) in self.vinculos.items():
            print(f"  {k}. {nombre_v} → ${precio:,}")

        while True:
            tipo_sel = input("Seleccione tipo (1-5): ").strip()
            if tipo_sel not in self.vinculos:
                print("Selección inválida.")
                continue
            tipo_vinculo, precio = self.vinculos[tipo_sel]
            break

        nuevo = Usuario(nombre, apellido, documento, tipo_vinculo, precio)
        self.usuarios.append(nuevo)
        print(f"Usuario {nuevo.nombre_completo()} registrado con éxito. Precio tiquete: ${nuevo.precio_tiquete:,}")

        # opcional: guardar ahora en CSV
        self.guardar_usuario_csv(nuevo)

    # ---------- registrar reserva ----------
    def registrar_reserva(self):
        print("\n--- Registrar Reserva ---")
        documento = input("Ingrese su documento: ").strip()
        usuario = self.buscar_usuario_por_documento(documento)
        if usuario is None:
            print("Usuario no encontrado. Debe registrarse primero.")
            choice = input("Desea registrarse ahora? (S/N): ").strip().upper()
            if choice == "S":
                self.registrar_usuario()
            return

        # Mostrar funciones disponibles con asiento libre
        print("\nFunciones disponibles:")
        lista_claves = []
        idx = 1
        for dia in self.dias:
            for pelicula, hora in self.peliculas:
                clave = self.key_from(dia, hora, pelicula)
                mapa = self.funciones[clave]
                libres = self.contar_asientos_libres(mapa)
                print(f"{idx}. {dia} - {hora} - {pelicula} - Asientos libres: {libres}")
                lista_claves.append((clave, dia, hora, pelicula))
                idx += 1

        sel = input("Seleccione el número de la función: ").strip()
        if not sel.isdigit() or not (1 <= int(sel) <= len(lista_claves)):
            print("Selección inválida.")
            return
        sel_idx = int(sel) - 1
        clave_funcion, dia, hora, pelicula = lista_claves[sel_idx]
        mapa = self.funciones[clave_funcion]

        # Mostrar mapa
        print(f"\nMapa de asientos para {dia} - {hora} - {pelicula}:")
        self.imprimir_mapa(mapa)

        # Pedir asiento
        asiento = input("Ingrese asiento (ej: B5): ").strip().upper()
        if not self.validar_asiento_str(asiento):
            print("Asiento inválido. Debe ser como B5 (A-K y 1-11).")
            return
        fila_idx, col_idx = self.asiento_to_indices(asiento)
        if mapa[fila_idx][col_idx] == "X":
            print("Asiento ocupado. Elija otro.")
            return

        # Reservar
        mapa[fila_idx][col_idx] = "X"
        fecha_res = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        precio = usuario.precio_tiquete
        reserva = Reserva(usuario.documento, dia, hora, pelicula, asiento, precio, fecha_res)
        self.reservas.append(reserva)
        print("\nReserva realizada con éxito. Generando factura...\n")
        self.imprimir_factura(usuario, reserva)
        # guardar reserva en CSV
        self.guardar_reserva_csv(reserva)

    # ---------- imprimir factura ----------
    def imprimir_factura(self, usuario, reserva):
        print("========== FACTURA - CineLab UDEA ==========")
        print(f"Nombre: {usuario.nombre_completo()}")
        print(f"Documento: {usuario.documento}")
        print(f"Película: {reserva.pelicula}")
        print(f"Día: {reserva.dia}")
        print(f"Hora: {reserva.hora}")
        print(f"Asiento: {reserva.asiento}")
        print(f"Precio: ${reserva.precio:,}")
        print(f"Total: ${reserva.precio:,}")
        print("-------------------------------------------")
        print("¡Gracias por usar CineLab UDEA!")
        print("===========================================\n")

    # ---------- cancelar reserva ----------
    def cancelar_reserva(self):
        print("\n--- Cancelar Reserva ---")
        documento = input("Ingrese su documento: ").strip()
        reservas_user = self.buscar_reservas_por_documento(documento)
        if not reservas_user:
            print("No tiene reservas activas.")
            choice = input("Desea hacer una reserva ahora? (S/N): ").strip().upper()
            if choice == "S":
                self.registrar_reserva()
            return

        print("\nSus reservas:")
        for i, r in enumerate(reservas_user, start=1):
            print(f"{i}. {r.dia} - {r.hora} - {r.pelicula} - Asiento: {r.asiento} (reservado: {r.fecha_reserva})")

        sel = input("Seleccione el número de la reserva a cancelar: ").strip()
        if not sel.isdigit() or not (1 <= int(sel) <= len(reservas_user)):
            print("Selección inválida.")
            return
        sel_idx = int(sel) - 1
        reserva = reservas_user[sel_idx]

        confirm = input("Confirma cancelar esta reserva? (S/N): ").strip().upper()
        if confirm != "S":
            print("Cancelación abortada.")
            return

        # Liberar asiento en mapa
        clave = self.key_from(reserva.dia, reserva.hora, reserva.pelicula)
        mapa = self.funciones.get(clave)
        if mapa:
            fi, ci = self.asiento_to_indices(reserva.asiento)
            mapa[fi][ci] = "0"

        # eliminar de reservas (la primera coincidencia)
        for i, r in enumerate(self.reservas):
            if (r.documento_usuario == reserva.documento_usuario and
                r.dia == reserva.dia and
                r.hora == reserva.hora and
                r.asiento == reserva.asiento):
                del self.reservas[i]
                break

        print("Reserva cancelada con éxito.")

    # ---------- consultar funciones ----------
    def consultar_funciones(self):
        print("\n--- Funciones del Fin de Semana ---\n")
        for dia in self.dias:
            print(f"== {dia} ==")
            for pelicula, hora in self.peliculas:
                clave = self.key_from(dia, hora, pelicula)
                mapa = self.funciones[clave]
                libres = self.contar_asientos_libres(mapa)
                print(f"{hora} - {pelicula} - Asientos libres: {libres}")
            print()
        # opcion para ver mapa de una funcion
        choice = input("Desea ver el mapa de una función específica? (S/N): ").strip().upper()
        if choice == "S":
            # listar con indices
            lista = []
            idx = 1
            for dia in self.dias:
                for pelicula, hora in self.peliculas:
                    lista.append((dia, hora, pelicula))
                    print(f"{idx}. {dia} - {hora} - {pelicula}")
                    idx += 1
            sel = input("Seleccione el número: ").strip()
            if sel.isdigit() and 1 <= int(sel) <= len(lista):
                sel_idx = int(sel) - 1
                dia, hora, pelicula = lista[sel_idx]
                clave = self.key_from(dia, hora, pelicula)
                print(f"\nMapa de asientos para {dia} - {hora} - {pelicula}:")
                self.imprimir_mapa(self.funciones[clave])
            else:
                print("Selección inválida.")
        print("Volviendo al menú principal...")

    # ---------- admin ----------
    def login_admin(self):
        print("\n--- Login Administrador ---")
        usuario = input("Usuario: ").strip()
        contr = input("Contraseña: ").strip()
        if self.admins.get(usuario) == contr:
            print("Acceso concedido.\n")
            self.menu_admin()
        else:
            print("Credenciales incorrectas.")

    def menu_admin(self):
        while True:
            print("\n--- Menú Administrador ---")
            print("1. Total de reservas registradas")
            print("2. Total de tiquetes vendidos")
            print("3. Total pago realizado")
            print("4. Promedio por venta diario (fin de semana)")
            print("5. Lista de usuarios")
            print("6. Usuario con mayor cantidad de reservas")
            print("7. Usuario con menor cantidad de reservas (con >0)")
            print("8. Exportar reportes a CSV")
            print("9. Volver al menú principal")
            op = input("Seleccione opción: ").strip()
            if op == "1":
                print(f"Total reservas registradas: {len(self.reservas)}")
            elif op == "2":
                print(f"Total tiquetes vendidos: {len(self.reservas)}")
            elif op == "3":
                total_pago = sum(r.precio for r in self.reservas)
                print(f"Total pago realizado: ${int(total_pago):,}")
            elif op == "4":
                total_pago = sum(r.precio for r in self.reservas)
                dias_operacion = len(self.dias)  # 3
                promedio = total_pago / dias_operacion if dias_operacion else 0
                print(f"Promedio por venta diario (fin de semana): ${int(promedio):,}")
            elif op == "5":
                if not self.usuarios:
                    print("No hay usuarios registrados.")
                else:
                    print("Usuarios registrados:")
                    for u in self.usuarios:
                        print(f"- {u.nombre_completo()} | Documento: {u.documento} | Vínculo: {u.tipo_vinculo}")
            elif op == "6":
                if not self.reservas:
                    print("No hay reservas.")
                else:
                    contador = {}
                    for r in self.reservas:
                        contador[r.documento_usuario] = contador.get(r.documento_usuario, 0) + 1
                    best_doc = max(contador, key=lambda k: contador[k])
                    user = self.buscar_usuario_por_documento(best_doc)
                    print(f"Usuario con mayor reservas: {user.nombre_completo()} | Cantidad: {contador[best_doc]}")
            elif op == "7":
                if not self.reservas:
                    print("No hay reservas.")
                else:
                    contador = {}
                    for r in self.reservas:
                        contador[r.documento_usuario] = contador.get(r.documento_usuario, 0) + 1
                    # filtrar >0 y minimizar
                    filtered = {k: v for k, v in contador.items() if v > 0}
                    if not filtered:
                        print("No hay usuarios con reservas.")
                    else:
                        min_doc = min(filtered, key=lambda k: filtered[k])
                        user = self.buscar_usuario_por_documento(min_doc)
                        print(f"Usuario con menor reservas (pero >0): {user.nombre_completo()} | Cantidad: {filtered[min_doc]}")
            elif op == "8":
                self.exportar_reportes_csv()
            elif op == "9":
                break
            else:
                print("Opción inválida.")

    # ---------- exportar / guardar CSV ----------
    def guardar_usuario_csv(self, usuario):
        # archivo usuarios.csv (append)
        try:
            with open("usuarios.csv", mode="a", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow([usuario.nombre, usuario.apellido, usuario.documento, usuario.tipo_vinculo, usuario.precio_tiquete])
        except Exception as e:
            print("Warning: no se pudo guardar usuarios.csv:", e)

    def guardar_reserva_csv(self, reserva):
        try:
            with open("reservas.csv", mode="a", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow([reserva.documento_usuario, reserva.dia, reserva.hora, reserva.pelicula, reserva.asiento, reserva.precio, reserva.fecha_reserva])
        except Exception as e:
            print("Warning: no se pudo guardar reservas.csv:", e)

    def exportar_reportes_csv(self):
        # Exportar usuarios
        try:
            with open("usuarios_full.csv", mode="w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(["Nombre", "Apellido", "Documento", "Vinculo", "Precio"])
                for u in self.usuarios:
                    writer.writerow([u.nombre, u.apellido, u.documento, u.tipo_vinculo, u.precio_tiquete])
            with open("reservas_full.csv", mode="w", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(["DocumentoUsuario", "Dia", "Hora", "Pelicula", "Asiento", "Precio", "FechaReserva"])
                for r in self.reservas:
                    writer.writerow([r.documento_usuario, r.dia, r.hora, r.pelicula, r.asiento, r.precio, r.fecha_reserva])
            print("Reportes exportados: usuarios_full.csv y reservas_full.csv")
        except Exception as e:
            print("Error exportando reportes:", e)

    # ---------- menu principal ----------
    def menu_principal(self):
        print(f"\nBienvenido a {self.nombre}\n")
        while True:
            print("1. Registrar Usuario")
            print("2. Registrar Reserva")
            print("3. Cancelar Reserva")
            print("4. Consultar Funciones Fin de Semana")
            print("5. Administrador")
            print("6. Salir")
            opcion = input("Seleccione una opción: ").strip()
            if opcion == "1":
                self.registrar_usuario()
            elif opcion == "2":
                self.registrar_reserva()
            elif opcion == "3":
                self.cancelar_reserva()
            elif opcion == "4":
                self.consultar_funciones()
            elif opcion == "5":
                self.login_admin()
            elif opcion == "6":
                print("Guardando datos y saliendo... Gracias por usar CineLab UDEA.")
                # intentar guardar todo al salir (usuarios y reservas)
                self.exportar_reportes_csv()
                break
            else:
                print("Opción inválida. Intente nuevamente.")

# ---------- Ejecutar programa ----------
if __name__ == "__main__":
    cine = Cinema()
    cine.menu_principal()


Bienvenido a CineLab UDEA

1. Registrar Usuario
2. Registrar Reserva
3. Cancelar Reserva
4. Consultar Funciones Fin de Semana
5. Administrador
6. Salir


KeyboardInterrupt: Interrupted by user